In [1]:
from evaluate import load
import csv
import pandas as pd
import numpy as np

# 文件路径
input_file = r'D:\liulanqi1\MindMap-main\MindMap-main\实验结果\output.csv'
output_file = r'D:\liulanqi1\MindMap-main\MindMap-main\实验结果\output_bert_score_multi_models_fixed.csv'

# 加载BERTScore评估器
bertscore = load("bertscore")

# 定义要使用的不同模型
models = [
    "distilbert-base-uncased",  # 原来使用的模型,       
    "xlm-roberta-large",             
    "roberta-large",          # 如果计算资源允许，可以取消注释使用这个更大的模型
]

def clean_text_for_csv(text):
    """清理文本，去除可能导致CSV解析问题的字符"""
    if not isinstance(text, str):
        return str(text)
    # 替换换行符为空格
    text = text.replace('\n', ' ')
    return text

# 使用pandas读取输入CSV文件
try:
    df = pd.read_csv(input_file)
    print(f"成功读取CSV文件，共有{len(df)}行")
    print(f"列名: {df.columns.tolist()}")
    
    # 为每个模型和每个输出创建新列
    for model_name in models:
        model_short_name = model_name.split("-")[0]  # 简化的模型名称
        for i in range(1, 7):
            df[f"{model_short_name}_output{i}_precision"] = np.nan
            df[f"{model_short_name}_output{i}_recall"] = np.nan
            df[f"{model_short_name}_output{i}_f1"] = np.nan
        # 为每个模型添加平均值列
        df[f"{model_short_name}_avg_precision"] = np.nan
        df[f"{model_short_name}_avg_recall"] = np.nan
        df[f"{model_short_name}_avg_f1"] = np.nan
    
    # 为每一行计算BERTScore
    for idx, row in df.iterrows():
        print(f"正在处理第 {idx+1}/{len(df)} 行...")
        reference = [clean_text_for_csv(row.iloc[1])]  # 假设参考文本在第2列
        
        # 为每个模型计算分数
        for model_name in models:
            model_short_name = model_name.split("-")[0]
            print(f"\n使用模型 {model_name} 计算BERTScore:")
            
            precision_values = []
            recall_values = []
            f1_values = []
            
            # 为每个输出计算BERTScore
            for i in range(1, 9):
                output_col_idx = i + 1  # 假设输出从第3列开始
                if output_col_idx < len(row):
                    output_text = [clean_text_for_csv(row.iloc[output_col_idx])]
                    
                    try:
                        results = bertscore.compute(
                            predictions=output_text, 
                            references=reference, 
                            model_type=model_name
                        )
                        
                        precision = results["precision"][0]
                        recall = results["recall"][0]
                        f1 = results["f1"][0]
                        
                        # 存储结果
                        df.at[idx, f"{model_short_name}_output{i}_precision"] = precision
                        df.at[idx, f"{model_short_name}_output{i}_recall"] = recall
                        df.at[idx, f"{model_short_name}_output{i}_f1"] = f1
                        
                        precision_values.append(precision)
                        recall_values.append(recall)
                        f1_values.append(f1)
                        
                        print(f"  Output{i} - P: {precision:.4f}, R: {recall:.4f}, F1: {f1:.4f}")
                    except Exception as e:
                        print(f"  计算Output{i}的BERTScore时出错: {e}")
                else:
                    print(f"  警告: 没有找到Output{i}的列")
            
            # 计算平均值
            if precision_values:
                avg_precision = np.mean(precision_values)
                avg_recall = np.mean(recall_values)
                avg_f1 = np.mean(f1_values)
                
                df.at[idx, f"{model_short_name}_avg_precision"] = avg_precision
                df.at[idx, f"{model_short_name}_avg_recall"] = avg_recall
                df.at[idx, f"{model_short_name}_avg_f1"] = avg_f1
                
                print(f"平均值 - P: {avg_precision:.4f}, R: {avg_recall:.4f}, F1: {avg_f1:.4f}")
            else:
                print("无法计算平均值，没有有效结果")
            
            print("-" * 50)
    
    # 保存结果到CSV文件
    df.to_csv(output_file, index=False, encoding='utf-8', quoting=csv.QUOTE_ALL)
    print(f"处理完成，结果已保存至 {output_file}")
    
except Exception as e:
    print(f"处理过程中出错: {e}")

成功读取CSV文件，共有29行
列名: ['Question', 'Label', 'MindMap', 'GPT3.5', 'BM25_retrieval', 'Embedding_retrieval', 'KG_retrieval', 'KG_self-consistency', 'KG_TreeOfThoughts', 'GPT4']
正在处理第 1/29 行...

使用模型 distilbert-base-uncased 计算BERTScore:


d:\python\Lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:402: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


  Output1 - P: 0.6351, R: 0.8641, F1: 0.7321
  Output2 - P: 0.8464, R: 0.8252, F1: 0.8356
  Output3 - P: 0.7548, R: 0.8423, F1: 0.7961
  Output4 - P: 0.7125, R: 0.8395, F1: 0.7708
  Output5 - P: 0.7009, R: 0.8173, F1: 0.7546
  Output6 - P: 0.8249, R: 0.8590, F1: 0.8416
  Output7 - P: 0.6582, R: 0.7995, F1: 0.7220
  Output8 - P: 0.7787, R: 0.8053, F1: 0.7918
平均值 - P: 0.7390, R: 0.8315, F1: 0.7806
--------------------------------------------------

使用模型 xlm-roberta-large 计算BERTScore:
  Output1 - P: 0.7770, R: 0.9210, F1: 0.8429
  Output2 - P: 0.8979, R: 0.8895, F1: 0.8937
  Output3 - P: 0.8455, R: 0.8983, F1: 0.8711
  Output4 - P: 0.8348, R: 0.8902, F1: 0.8616
  Output5 - P: 0.8224, R: 0.8856, F1: 0.8528
  Output6 - P: 0.8839, R: 0.9111, F1: 0.8973
  Output7 - P: 0.8002, R: 0.8780, F1: 0.8373
  Output8 - P: 0.8704, R: 0.8771, F1: 0.8737
平均值 - P: 0.8415, R: 0.8939, F1: 0.8663
--------------------------------------------------

使用模型 roberta-large 计算BERTScore:


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  Output1 - P: 0.7504, R: 0.9041, F1: 0.8201
  Output2 - P: 0.8982, R: 0.8736, F1: 0.8857
  Output3 - P: 0.8329, R: 0.8870, F1: 0.8591
  Output4 - P: 0.8207, R: 0.8768, F1: 0.8479
  Output5 - P: 0.8079, R: 0.8615, F1: 0.8338
  Output6 - P: 0.8585, R: 0.8967, F1: 0.8772
  Output7 - P: 0.7770, R: 0.8511, F1: 0.8124
  Output8 - P: 0.8675, R: 0.8617, F1: 0.8645
平均值 - P: 0.8266, R: 0.8766, F1: 0.8501
--------------------------------------------------
正在处理第 2/29 行...

使用模型 distilbert-base-uncased 计算BERTScore:
  Output1 - P: 0.6710, R: 0.8355, F1: 0.7443
  Output2 - P: 0.7827, R: 0.8555, F1: 0.8175
  Output3 - P: 0.7066, R: 0.8188, F1: 0.7585
  Output4 - P: 0.7316, R: 0.8340, F1: 0.7795
  Output5 - P: 0.7148, R: 0.8276, F1: 0.7670
  Output6 - P: 0.8056, R: 0.8684, F1: 0.8358
  Output7 - P: 0.6819, R: 0.8297, F1: 0.7486
  Output8 - P: 0.7059, R: 0.8329, F1: 0.7641
平均值 - P: 0.7250, R: 0.8378, F1: 0.7769
--------------------------------------------------

使用模型 xlm-roberta-large 计算BERTScore:
  Ou

...

  Output1 - P: 0.7461, R: 0.8678, F1: 0.8023
  Output2 - P: 0.8595, R: 0.8579, F1: 0.8587
  Output3 - P: 0.8174, R: 0.8618, F1: 0.8391
  Output4 - P: 0.8076, R: 0.8678, F1: 0.8366
  Output5 - P: 0.8329, R: 0.8668, F1: 0.8495
  Output6 - P: 0.8690, R: 0.8935, F1: 0.8811
  Output7 - P: 0.7689, R: 0.8622, F1: 0.8129
  Output8 - P: 0.8468, R: 0.8586, F1: 0.8526
平均值 - P: 0.8185, R: 0.8670, F1: 0.8416
--------------------------------------------------
正在处理第 28/29 行...

使用模型 distilbert-base-uncased 计算BERTScore:
  Output1 - P: 0.6004, R: 0.8242, F1: 0.6947
  Output2 - P: 0.7798, R: 0.8270, F1: 0.8027
  Output3 - P: 0.7431, R: 0.8286, F1: 0.7835
  Output4 - P: 0.7292, R: 0.8144, F1: 0.7694
  Output5 - P: 0.6890, R: 0.8182, F1: 0.7481
  Output6 - P: 0.7707, R: 0.8457, F1: 0.8065
  Output7 - P: 0.6516, R: 0.8187, F1: 0.7257
  Output8 - P: 0.6715, R: 0.8116, F1: 0.7350
平均值 - P: 0.7044, R: 0.8236, F1: 0.7582
--------------------------------------------------

使用模型 xlm-roberta-large 计算BERTScore:
  O

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  Output1 - P: 0.7275, R: 0.8762, F1: 0.7949
  Output2 - P: 0.8590, R: 0.8674, F1: 0.8631
  Output3 - P: 0.8319, R: 0.8748, F1: 0.8528
  Output4 - P: 0.8304, R: 0.8767, F1: 0.8529
  Output5 - P: 0.8023, R: 0.8710, F1: 0.8352
  Output6 - P: 0.8429, R: 0.9035, F1: 0.8721
  Output7 - P: 0.7759, R: 0.8723, F1: 0.8213
  Output8 - P: 0.7964, R: 0.8616, F1: 0.8277
平均值 - P: 0.8083, R: 0.8754, F1: 0.8400
--------------------------------------------------
正在处理第 29/29 行...

使用模型 distilbert-base-uncased 计算BERTScore:
  Output1 - P: 0.6460, R: 0.7790, F1: 0.7063
  Output2 - P: 0.7863, R: 0.7872, F1: 0.7868
  Output3 - P: 0.7316, R: 0.7940, F1: 0.7615
  Output4 - P: 0.7395, R: 0.7798, F1: 0.7592
  Output5 - P: 0.7192, R: 0.7828, F1: 0.7496
  Output6 - P: 0.8031, R: 0.8001, F1: 0.8016
  Output7 - P: 0.6468, R: 0.8014, F1: 0.7158
  Output8 - P: 0.7051, R: 0.7910, F1: 0.7456
平均值 - P: 0.7222, R: 0.7894, F1: 0.7533
--------------------------------------------------

使用模型 xlm-roberta-large 计算BERTScore:
  O

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  Output1 - P: 0.7384, R: 0.8615, F1: 0.7952
  Output2 - P: 0.8580, R: 0.8671, F1: 0.8625
  Output3 - P: 0.8205, R: 0.8653, F1: 0.8423
  Output4 - P: 0.8363, R: 0.8615, F1: 0.8487
  Output5 - P: 0.8146, R: 0.8676, F1: 0.8403
  Output6 - P: 0.8642, R: 0.8806, F1: 0.8723
  Output7 - P: 0.7764, R: 0.8614, F1: 0.8167
  Output8 - P: 0.8143, R: 0.8560, F1: 0.8346
平均值 - P: 0.8153, R: 0.8651, F1: 0.8391
--------------------------------------------------
处理完成，结果已保存至 D:\liulanqi1\MindMap-main\MindMap-main\实验结果\output_bert_score_multi_models_fixed.csv


In [2]:
import pandas as pd
import numpy as np

input_file = r'D:\liulanqi1\MindMap-main\MindMap-main\实验结果\output_bert_score_multi_models_fixed.csv'
output_file = r'D:\liulanqi1\MindMap-main\MindMap-main\实验结果\bertscore_output_averages.csv'

# 定义 output 对应的真实名称
output_name_map = {
    'output1': 'MindMap',
    'output2': 'GPT3.5',
    'output3': 'BM25_retrieval',
    'output4': 'Embedding_retrieval',
    'output5': 'KG_retrieval',
    'output6': 'KG_self-consistency',
    'output7': 'KG_TreeOfThoughts',
    'output8': 'GPT4'
}

try:
    df = pd.read_csv(input_file)
    print(f"成功读取CSV文件，共有{len(df)}行")

    # Detect model prefixes from column names
    model_prefixes = set()
    for col in df.columns:
        if '_output' in col and ('_precision' in col or '_recall' in col or '_f1' in col):
            prefix = col.split('_output', 1)[0]
            model_prefixes.add(prefix)

    print(f"检测到的模型: {sorted(list(model_prefixes))}")

    # 1. Calculate average score ACROSS MODELS for each output
    results = []
    for i in range(1, 9):
        output_metrics = {'output_number': f'output{i}', 'avg_precision': 0.0, 'avg_recall': 0.0, 'avg_f1': 0.0, 'count': 0}
        for prefix in model_prefixes:
            precision_col = f"{prefix}_output{i}_precision"
            recall_col = f"{prefix}_output{i}_recall"
            f1_col = f"{prefix}_output{i}_f1"

            if all(col in df.columns for col in [precision_col, recall_col, f1_col]):
                avg_precision = df[precision_col].mean(skipna=True)
                avg_recall = df[recall_col].mean(skipna=True)
                avg_f1 = df[f1_col].mean(skipna=True)

                if not np.isnan(avg_precision) and not np.isnan(avg_recall) and not np.isnan(avg_f1):
                    output_metrics['avg_precision'] += avg_precision
                    output_metrics['avg_recall'] += avg_recall
                    output_metrics['avg_f1'] += avg_f1
                    output_metrics['count'] += 1

        if output_metrics['count'] > 0:
            for metric in ['avg_precision', 'avg_recall', 'avg_f1']:
                output_metrics[metric] /= output_metrics['count']
            results.append(output_metrics)

    # 2. Calculate OVERALL average across all models and outputs
    if results:
        overall_avg = {
            'output_number': 'overall',
            'avg_precision': np.mean([r['avg_precision'] for r in results]),
            'avg_recall': np.mean([r['avg_recall'] for r in results]),
            'avg_f1': np.mean([r['avg_f1'] for r in results])
        }
        results.append(overall_avg)

    # 3. Per-model averages + detailed per-output per-model
    model_results = []
    detailed_model_output_results = []

    for prefix in sorted(list(model_prefixes)):
        model_metrics = {'model': prefix, 'avg_precision': 0.0, 'avg_recall': 0.0, 'avg_f1': 0.0, 'output_count': 0}
        for i in range(1, 9):
            precision_col = f"{prefix}_output{i}_precision"
            recall_col = f"{prefix}_output{i}_recall"
            f1_col = f"{prefix}_output{i}_f1"

            if all(col in df.columns for col in [precision_col, recall_col, f1_col]):
                avg_precision = df[precision_col].mean(skipna=True)
                avg_recall = df[recall_col].mean(skipna=True)
                avg_f1 = df[f1_col].mean(skipna=True)

                if not np.isnan(avg_precision) and not np.isnan(avg_recall) and not np.isnan(avg_f1):
                    detailed_model_output_results.append({
                        'model': prefix,
                        'output_number': f'output{i}',
                        'avg_precision': avg_precision,
                        'avg_recall': avg_recall,
                        'avg_f1': avg_f1
                    })

                    model_metrics['avg_precision'] += avg_precision
                    model_metrics['avg_recall'] += avg_recall
                    model_metrics['avg_f1'] += avg_f1
                    model_metrics['output_count'] += 1

        if model_metrics['output_count'] > 0:
            for metric in ['avg_precision', 'avg_recall', 'avg_f1']:
                model_metrics[metric] /= model_metrics['output_count']
            del model_metrics['output_count']
            model_results.append(model_metrics)

    # Save combined results
    final_df = pd.concat([pd.DataFrame(results), pd.DataFrame(model_results)], ignore_index=True)
    final_df.to_csv(output_file, index=False, float_format='%.6f')
    print(f"\n聚合平均结果已保存到 {output_file}")

    # --- Printing Tables ---

    # Table 1: Sorted by avg_f1
    print("\n各输出的平均BERTScore (按F1从高到低排序):")
    print("=" * 80)
    print(f"{'输出名称':^25} | {'平均Precision':^15} | {'平均Recall':^15} | {'平均F1':^15}")
    print("-" * 80)

    sorted_results = sorted(
        [r for r in results if r['output_number'] != 'overall'],
        key=lambda x: x['avg_f1'],
        reverse=True
    )

    for r in sorted_results:
        output_number = r.get('output_number', 'N/A')
        display_name = output_name_map.get(output_number, output_number) + f" ({output_number})"
        print(f"{display_name:<25} | {r.get('avg_precision', 0):^15.4f} | {r.get('avg_recall', 0):^15.4f} | {r.get('avg_f1', 0):^15.4f}")

    # Print overall row
    overall_row = next((r for r in results if r['output_number'] == 'overall'), None)
    if overall_row:
        print("-" * 80)
        print(f"{'OVERALL':<25} | {overall_row.get('avg_precision', 0):^15.4f} | {overall_row.get('avg_recall', 0):^15.4f} | {overall_row.get('avg_f1', 0):^15.4f}")
    print("-" * 80)

    # Table 2: Average per model
    print("\n各模型的平均BERTScore (所有输出平均):")
    print("=" * 65)
    print(f"{'模型':^20} | {'平均Precision':^15} | {'平均Recall':^15} | {'平均F1':^15}")
    print("-" * 65)
    model_results_print_df = pd.DataFrame(model_results).fillna(0)
    for index, m in model_results_print_df.iterrows():
        print(f"{m.get('model', 'N/A'):<20} | {m.get('avg_precision', 0):^15.4f} | {m.get('avg_recall', 0):^15.4f} | {m.get('avg_f1', 0):^15.4f}")
    print("-" * 65)

    # Table 3: Detailed per model and output, sorted by F1
    print("\n各模型在各输出上的详细平均BERTScore (按F1从高到低排序):")
    print("=" * 100)
    print(f"{'模型':^20} | {'输出名称':^25} | {'平均Precision':^15} | {'平均Recall':^15} | {'平均F1':^15}")
    print("-" * 100)
    detailed_df = pd.DataFrame(detailed_model_output_results)
    if not detailed_df.empty:
        detailed_df = detailed_df.fillna(0)
        # 按 avg_f1 从高到低排序
        detailed_df = detailed_df.sort_values(by=['avg_f1'], ascending=False)
        for _, d in detailed_df.iterrows():
            output_number = d.get('output_number', 'N/A')
            display_name = output_name_map.get(output_number, output_number) + f" ({output_number})"
            print(f"{d.get('model', 'N/A'):<20} | {display_name:<25} | {d.get('avg_precision', 0):^15.4f} | {d.get('avg_recall', 0):^15.4f} | {d.get('avg_f1', 0):^15.4f}")
        print("-" * 100)
    else:
        print("⚠ 没有可用的详细模型输出数据，跳过详细表格打印。")

except FileNotFoundError:
    print(f"错误: 输入文件 '{input_file}' 未找到。")
except Exception as e:
    print(f"处理过程中发生错误: {e}")
    import traceback
    traceback.print_exc()


成功读取CSV文件，共有29行
检测到的模型: ['distilbert', 'roberta', 'xlm']

聚合平均结果已保存到 D:\liulanqi1\MindMap-main\MindMap-main\实验结果\bertscore_output_averages.csv

各输出的平均BERTScore (按F1从高到低排序):
          输出名称            |   平均Precision   |    平均Recall     |      平均F1      
--------------------------------------------------------------------------------
KG_self-consistency (output6) |     0.8316      |     0.8577      |     0.8443     
GPT3.5 (output2)          |     0.8263      |     0.8387      |     0.8321     
Embedding_retrieval (output4) |     0.7930      |     0.8467      |     0.8187     
BM25_retrieval (output3)  |     0.7900      |     0.8429      |     0.8153     
KG_retrieval (output5)    |     0.7814      |     0.8451      |     0.8116     
GPT4 (output8)            |     0.7814      |     0.8407      |     0.8095     
KG_TreeOfThoughts (output7) |     0.7363      |     0.8341      |     0.7815     
MindMap (output1)         |     0.7075      |     0.8463      |     0.7701     
----------------

In [3]:
import pandas as pd
import numpy as np

input_file = r'D:\liulanqi1\MindMap-main\MindMap-main\实验结果\output_bert_score_multi_models_fixed.csv'
output_file_base = r'D:\liulanqi1\MindMap-main\MindMap-main\实验结果'

# 定义 output 对应的真实名称
output_name_map = {
    'output1': 'MindMap',
    'output2': 'GPT3.5',
    'output3': 'BM25_retrieval',
    'output4': 'Embedding_retrieval',
    'output5': 'KG_retrieval',
    'output6': 'KG_self-consistency',
    'output7': 'KG_TreeOfThoughts',
    'output8': 'GPT4'
}

try:
    df = pd.read_csv(input_file)
    print(f"成功读取CSV文件，共有{len(df)}行")

    model_prefixes = set()
    for col in df.columns:
        if '_output' in col and ('_precision' in col or '_recall' in col or '_f1' in col):
            prefix = col.split('_output', 1)[0]
            model_prefixes.add(prefix)

    print(f"检测到的模型: {sorted(list(model_prefixes))}")

    # 1. Calculate average score ACROSS MODELS for each output
    results = []
    for i in range(1, 9):
        output_metrics = {'output_number': f'output{i}', 'output_name': output_name_map.get(f'output{i}', f'output{i}'),
                          'avg_precision': 0.0, 'avg_recall': 0.0, 'avg_f1': 0.0, 'count': 0}
        for prefix in model_prefixes:
            precision_col = f"{prefix}_output{i}_precision"
            recall_col = f"{prefix}_output{i}_recall"
            f1_col = f"{prefix}_output{i}_f1"

            if all(col in df.columns for col in [precision_col, recall_col, f1_col]):
                avg_precision = df[precision_col].mean(skipna=True)
                avg_recall = df[recall_col].mean(skipna=True)
                avg_f1 = df[f1_col].mean(skipna=True)

                if not np.isnan(avg_precision) and not np.isnan(avg_recall) and not np.isnan(avg_f1):
                    output_metrics['avg_precision'] += avg_precision
                    output_metrics['avg_recall'] += avg_recall
                    output_metrics['avg_f1'] += avg_f1
                    output_metrics['count'] += 1

        if output_metrics['count'] > 0:
            for metric in ['avg_precision', 'avg_recall', 'avg_f1']:
                output_metrics[metric] /= output_metrics['count']
            results.append(output_metrics)

    # 2. Calculate OVERALL average
    if results:
        overall_avg = {
            'output_number': 'overall',
            'output_name': 'Overall',
            'avg_precision': np.mean([r['avg_precision'] for r in results]),
            'avg_recall': np.mean([r['avg_recall'] for r in results]),
            'avg_f1': np.mean([r['avg_f1'] for r in results])
        }
        results.append(overall_avg)

    # 3. Per-model averages + detailed per-output per-model
    model_results = []
    detailed_model_output_results = []

    for prefix in sorted(list(model_prefixes)):
        model_metrics = {'model': prefix, 'avg_precision': 0.0, 'avg_recall': 0.0, 'avg_f1': 0.0, 'output_count': 0}
        for i in range(1, 9):
            precision_col = f"{prefix}_output{i}_precision"
            recall_col = f"{prefix}_output{i}_recall"
            f1_col = f"{prefix}_output{i}_f1"

            if all(col in df.columns for col in [precision_col, recall_col, f1_col]):
                avg_precision = df[precision_col].mean(skipna=True)
                avg_recall = df[recall_col].mean(skipna=True)
                avg_f1 = df[f1_col].mean(skipna=True)

                if not np.isnan(avg_precision) and not np.isnan(avg_recall) and not np.isnan(avg_f1):
                    detailed_model_output_results.append({
                        'model': prefix,
                        'output_number': f'output{i}',
                        'output_name': output_name_map.get(f'output{i}', f'output{i}'),
                        'avg_precision': avg_precision,
                        'avg_recall': avg_recall,
                        'avg_f1': avg_f1
                    })

                    model_metrics['avg_precision'] += avg_precision
                    model_metrics['avg_recall'] += avg_recall
                    model_metrics['avg_f1'] += avg_f1
                    model_metrics['output_count'] += 1

        if model_metrics['output_count'] > 0:
            for metric in ['avg_precision', 'avg_recall', 'avg_f1']:
                model_metrics[metric] /= model_metrics['output_count']
            del model_metrics['output_count']
            model_results.append(model_metrics)

    # === Save sorted outputs (by F1) ===
    outputs_df = pd.DataFrame([r for r in results if r['output_number'] != 'overall'])
    outputs_df_sorted = outputs_df.sort_values(by='avg_f1', ascending=False)
    outputs_df_sorted.to_csv(f"{output_file_base}\\outputs_sorted.csv", index=False, float_format='%.6f')
    print(f"已保存按F1排序的输出到 {output_file_base}\\outputs_sorted.csv")

    # === Save overall model averages ===
    model_df = pd.DataFrame(model_results)
    model_df.to_csv(f"{output_file_base}\\models_average.csv", index=False, float_format='%.6f')
    print(f"已保存模型平均值到 {output_file_base}\\models_average.csv")

    # === Save detailed per-model per-output sorted by F1 ===
    detailed_df = pd.DataFrame(detailed_model_output_results)
    detailed_df_sorted = detailed_df.sort_values(by='avg_f1', ascending=False)
    detailed_df_sorted.to_csv(f"{output_file_base}\\detailed_sorted.csv", index=False, float_format='%.6f')
    print(f"已保存按F1排序的详细表到 {output_file_base}\\detailed_sorted.csv")

except FileNotFoundError:
    print(f"错误: 输入文件 '{input_file}' 未找到。")
except Exception as e:
    print(f"处理过程中发生错误: {e}")
    import traceback
    traceback.print_exc()


成功读取CSV文件，共有29行
检测到的模型: ['distilbert', 'roberta', 'xlm']
已保存按F1排序的输出到 D:\liulanqi1\MindMap-main\MindMap-main\实验结果\outputs_sorted.csv
已保存模型平均值到 D:\liulanqi1\MindMap-main\MindMap-main\实验结果\models_average.csv
已保存按F1排序的详细表到 D:\liulanqi1\MindMap-main\MindMap-main\实验结果\detailed_sorted.csv
